In [11]:
from PIL import Image
import pdf2image
import imutils
from imutils import contours
from pytesseract import image_to_string, image_to_osd
import cv2
import os
import numpy as np

## Reading of the image and its rotating

In [4]:
def rotate_passport(image):
    
    # Initializing cascade
    cascade = cv2.CascadeClassifier('cascade.xml')
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    
    rotates = 0
    # Looking for a face
    for _ in range(4):
        
        face = cascade.detectMultiScale(gray, 1.3, 5)
        
        if face is not ():
            return imutils.rotate_bound(image, 90 * rotates)
        
        gray = imutils.rotate_bound(gray, 90)
        rotates += 1
    
    # Return false if the given picture is not a passport
    return False

In [5]:
def cut_passport(image):
    
    # Initializing cascade
    cascade = cv2.CascadeClassifier('cascade.xml')
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    
    # Finding a face
    face = cascade.detectMultiScale(gray, 1.3, 5)
    
    # Cutting the image so only passport was left
    (x, y, w, h) = face[0]
    
    output = image.copy()
    cv2.rectangle(output, (x, y), (x + w, y + h),
                  (0, 0, 255), 2)

    image = image[y - 6 * h: y + 3 * h, x - w:x + 6 * w]
    
    return image

In [59]:
name = 'test/224.jpeg'
image = cv2.imread(name)

image = rotate_passport(image)
image = cut_passport(image)
image = imutils.resize(image, width=750)

cv2.imshow('Output', image)
cv2.waitKey(0)

-1

## Reading text

In [60]:
def read_text_from_box(image):
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    
    kernel = np.ones((1, 1), np.uint8)
    gray = cv2.dilate(gray, kernel, iterations=1)
    gray = cv2.erode(gray, kernel, iterations=1)

    blurred = cv2.GaussianBlur(gray.copy(), (5, 5), 0)
    thresh = cv2.threshold(blurred, 0, 225, \
                           cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    cv2.imshow('Output', image)
    cv2.waitKey(0)

    text = image_to_string(thresh, lang='rus').replace('\n', ' ')
    
    return text

In [107]:
def locate_text(image):
    
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))
    sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (12, 12))
    
    #image = imutils.resize(image, width=300)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    tophat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKernel)
    
    cv2.imshow('Tophat', tophat)
    cv2.waitKey(0)
    
    gradX = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0,
                    ksize=-1)
    gradX = np.absolute(gradX)
    (minVal, maxVal) = (np.min(gradX), np.max(gradX))
    gradX = (255 * ((gradX - minVal) / (maxVal - minVal)))
    gradX = gradX.astype("uint8")
    
    cv2.imshow('Gradient', gradX)
    cv2.waitKey(0)
    
    gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)
    thresh = cv2.threshold(gradX, 0, 255,
                cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
    
    p = int(image.shape[1] * 0.05)
    thresh[:, 0:p] = 0
    thresh[:, image.shape[1] - p:] = 0

    cv2.imshow('Thresh', thresh)
    cv2.waitKey(0)
    
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    locs = []
    
    for (i, c) in enumerate(cnts):
        (x, y, w, h) = cv2.boundingRect(c)
        ar = w / float(h)
        
        if w > 10 and h > 10 and ar > 2.5:
            locs.append((x, y, w, h))
    
    locs = sorted(locs, key=lambda x:x[0])
    output = []
    
    for (i, (gX, gY, gW, gH)) in enumerate(locs):
        groupOutput = []
 

        group = gray[gY - 5:gY + gH + 5, gX - 5:gX + gW + 5]
        group = cv2.threshold(group, 0, 255,
                    cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
 
        digitCnts = cv2.findContours(group.copy(), cv2.RETR_EXTERNAL,
                    cv2.CHAIN_APPROX_SIMPLE)
        digitCnts = imutils.grab_contours(digitCnts)
        digitCnts = contours.sort_contours(digitCnts,
                    method="left-to-right")[0]
        
        cv2.rectangle(image, (gX - 5, gY - 5),
                        (gX + gW + 5, gY + gH + 5), (0, 0, 255), 2)
        
        cv2.imshow('ROI', image)
        cv2.waitKey(0)
        #cv2.destroyAllWindows()
        
        """
        for c in digitCnts:
            (x, y, w, h) = cv2.boundingRect(c)
            roi = group[y:y + h, x:x + w]
            roi = cv2.resize(roi, (57, 88))
 
            scores = []

            for (digit, digitROI) in digits.items():
            
                result = cv2.matchTemplate(roi, digitROI,
                    cv2.TM_CCOEFF)
                (_, score, _, _) = cv2.minMaxLoc(result)
                scores.append(score)


            groupOutput.append(str(np.argmax(scores)))
        """

In [120]:
name = 'test/249.jpeg'
image = cv2.imread(name)

image = rotate_passport(image)
image = cut_passport(image)
image = imutils.resize(image, width=750)

(h, w, _) = image.shape
box = image[int(h/2):h, int(w/3):w]
locate_text(box)
bottom = read_text_from_box(box)

In [121]:
name = 'test/249.jpeg'
image = cv2.imread(name)

image = rotate_passport(image)
image = cut_passport(image)
image = imutils.resize(image, width=750)

box = image[0:int(h/3), 0:w]
locate_text(box)
top = read_text_from_box(box)

In [21]:
image_ = imutils.rotate_bound(image, -90)
(h, w, _) = image_.shape
box = image_[0:int(h/10), 0:w]
number = read_text_from_box(box)

AttributeError: 'NoneType' object has no attribute 'copy'

## Classifing text

In [259]:
passport = {
    'ocr_result': {
        'doc_type': 'passport',
        'issue_authority': '',
        'issue_code': '',
        'issue_date': '',
        'surname': '',
        'name': '',
        'patronymic_name': '',
        'birth_date': '',
        'gender': '',
        'birth_place': '',
        'series': '',
        'number': '',
    },
    'text': '',
}

In [260]:
passport['text'] = top + bottom

In [261]:
import re

In [262]:
# Looking for issue code
code = re.search(r'\d{3}-\d{3}', top)
if code is not None:
    passport['ocr_result']['issue_code'] = code[0]

# Looking for issue authority

AUTHORITIES = ['отделом', 'УФМС', 'МФЦ', 'УВД']

authority = ''

for auth in AUTHORITIES:
    if re.search(auth, top, flags=re.I) is not None:
        
        issued = re.search(r'{}.*'.format(auth), top, flags=re.I)[0].split(' ')
        for i in issued:
            if all(c.isupper() or c == '-' or c == '.' for c in i):
                authority += i + ' '
        break
                
    
# If nothing that could be related to issue authority was found,
# consider all words in the top to be such
if authority is '':
    
    for word in top.split():
        if all(c.isalpha() or c == '-' or c == '.' for c in word) \
                    and len(word) > 3:
            authority += word + ' '
            
    
passport['ocr_result']['issue_authority'] = authority
    
# Looking for issue date
date = re.search(r'\d{2}\.\d{2}\.\d{4}', top)
if date is not None:
    passport['ocr_result']['issue_date'] = date[0]

# Looking for name
full_name = re.search(r'(.*( .*(ВИЧ|ВНА)))', bottom, flags=re.I)

if full_name is not None:

    passport['ocr_result']['patronymic_name'] = full_name[2]
    full_name = full_name[0].split()

    name = []
    for n in full_name:
        if all(c.isalpha() for c in n) and len(n) >= 3:
            name.append(n)
        
    if len(name) > 1:
        passport['ocr_result']['surname'] = name[0]
        passport['ocr_result']['name'] = name[1]
        

# Looking for birth date
date = re.search(r'\d{2}\.\d{2}\.\d{4}', bottom)
if date is not None:
    passport['ocr_result']['birth_date'] = date[0]
    
# Looking for gender

if passport['ocr_result']['patronymic_name'].endswith('ВИЧ') \
                    or re.search(r'(МУЖ|МУЖ.) ', bottom) is not None:
    passport['ocr_result']['gender'] = 'male'
elif passport['ocr_result']['patronymic_name'].endswith('ВНА') \
                    or re.search(r'(ЖЕН|ЖЕН.) ', bottom) is not None:
    passport['ocr_result']['gender'] = 'female'
    
# Looking for place of birth

genders = ['МУЖ', 'МУЖ.', 'ЖЕН', 'ЖЕН.']
birth_place = ''
for word in bottom.split():
    if all(c.isalpha() or c == '.' for c in word) and word not in name  \
                            and word not in genders and len(word) > 2:
        birth_place += word + ' '
passport['ocr_result']['birth_place'] = birth_place
    
# Looking for passport series

series = re.search(r'(\d{2} \d{2})', number)
if series is not None:
    passport['ocr_result']['series'] = series[0]
    
# Looking for passport number

num = re.search(r'(\d{6})', number)
if num is not None:
    passport['ocr_result']['number'] = num[0] 

In [263]:
passport

{'ocr_result': {'doc_type': 'passport',
  'issue_authority': 'ОТДЕЛОМ УФМС РОССИИ ПО ХАНТЫ-МАНСИЙСКОМУ АВТОНОМ. ОКР. -ЮГРЕ В ГОРОДЕ СУРГУТЕ       ',
  'issue_code': '860-004',
  'issue_date': '22.05.2011',
  'surname': 'ЛЫСЕНКО',
  'name': 'наталья',
  'patronymic_name': ' АЛЕКСАНДРОВНА',
  'birth_date': '43.04.1985',
  'gender': 'female',
  'birth_place': 'СУРГУТ ТЮМЕНСКОЙ ОБЛАСТИ ',
  'series': '',
  'number': ''},
 'text': 'Ирод ОТДЕЛОМ УФМС РОССИИ ПО ХАНТЫ-МАНСИЙСКОМУ АВТОНОМ. ОКР. -ЮГРЕ В ГОРОДЕ СУРГУТЕ 22.05.2011 860-004  Аата кылачи Кол подразделения  Аннный кол     / 1ЛЫСЕНКО ©  наталья аи АЛЕКСАНДРОВНА | „. ЖЕН. “, 43.04.1985 |  `." „ГОР. СУРГУТ ТЮМЕНСКОЙ ОБЛАСТИ  р --'}

In [252]:
full_name = re.search(r'(.*( .*(ВИЧ|ВНА)))', bottom, flags=re.I)

## EAST Text Detection

In [351]:
from imutils.object_detection import non_max_suppression

In [352]:
image = box.copy()

gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)

kernel = np.ones((1, 1), np.uint8)
gray = cv2.dilate(gray, kernel, iterations=1)
gray = cv2.erode(gray, kernel, iterations=1)

blurred = cv2.GaussianBlur(gray.copy(), (5, 5), 0)
thresh = cv2.threshold(blurred, 0, 225, \
                       cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

cv2.imshow('Output', thresh)
cv2.waitKey(0)

-1

In [365]:
orig = image.copy()
(H, W) = image.shape[:2]
 
(newW, newH) = (640, 640)
rW = W / float(newW)
rH = H / float(newH)
 
image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]

In [366]:
layerNames = [
    "feature_fusion/Conv_7/Sigmoid",
    "feature_fusion/concat_3"]

In [367]:
net = cv2.dnn.readNet('frozen_east_text_detection.pb')

In [368]:
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
    (123.68, 116.78, 103.94), swapRB=True, crop=False)
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)

In [369]:
(numRows, numCols) = scores.shape[2:4]
rects = []
confidences = []
 
for y in range(0, numRows):
    scoresData = scores[0, 0, y]
    xData0 = geometry[0, 0, y]
    xData1 = geometry[0, 1, y]
    xData2 = geometry[0, 2, y]
    xData3 = geometry[0, 3, y]
    anglesData = geometry[0, 4, y]
    for x in range(0, numCols):
        # if our score does not have sufficient probability, ignore it
        if scoresData[x] < 0.9:
            continue

        # compute the offset factor as our resulting feature maps will
        # be 4x smaller than the input image
        (offsetX, offsetY) = (x * 4.0, y * 4.0)

        # extract the rotation angle for the prediction and then
        # compute the sin and cosine
        angle = anglesData[x]
        cos = np.cos(angle)
        sin = np.sin(angle)

        # use the geometry volume to derive the width and height of
        # the bounding box
        h = xData0[x] + xData2[x]
        w = xData1[x] + xData3[x]

        # compute both the starting and ending (x, y)-coordinates for
        # the text prediction bounding box
        endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
        endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
        startX = int(endX - w)
        startY = int(endY - h)

        # add the bounding box coordinates and probability score to
        # our respective lists
        rects.append((startX, startY, endX, endY))
        confidences.append(scoresData[x])

In [371]:
boxes = non_max_suppression(np.array(rects), probs=confidences)

for (startX, startY, endX, endY) in boxes:
    startX = int(startX * rW)
    startY = int(startY * rH)
    endX = int(endX * rW)
    endY = int(endY * rH)

    cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2)

cv2.imshow("Text Detection", orig)
cv2.waitKey(0)

-1